# 📋 GERADOR DE DATA-SCHEMAS - ARGOS SETORES

Este notebook gera automaticamente os arquivos de data-schema para todas as tabelas do projeto.

**O que será gerado:**
- `DESCRIBE FORMATTED` para cada tabela
- `SELECT * FROM ... LIMIT 10` para cada tabela
- Arquivos markdown organizados em `./data-schema/`

**Tabelas processadas:**
- 3 tabelas originais (ODS)
- 9 tabelas intermediárias (NIAT)
- 4 views auxiliares (opcional)

---

## 🔧 PASSO 1: Configurar Sessão Spark

Execute esta célula primeiro para configurar a sessão Spark.

In [ ]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

from utils import spark_utils_session as utils
from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    app_name = "tsevero_data_schema_generator"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')
spark = session.sparkSession

print("✅ Sessão Spark configurada com sucesso!")
print(f"App Name: {spark.sparkContext.appName}")

## 📦 PASSO 2: Carregar Script Gerador

Carrega as funções do gerador de data-schemas.

In [ ]:
import os
from datetime import datetime
from pathlib import Path
import pandas as pd

# =============================================================================
# CONFIGURAÇÃO
# =============================================================================

TABELAS_ORIGINAIS = [
    "usr_sat_ods.vw_ods_decl_dime",
    "usr_sat_ods.vw_ods_contrib",
    "usr_sat_ods.vw_ods_pagamento"
]

TABELAS_INTERMEDIARIAS = [
    "niat.argos_benchmark_setorial",
    "niat.argos_benchmark_setorial_porte",
    "niat.argos_empresas",
    "niat.argos_pagamentos_empresa",
    "niat.argos_empresa_vs_benchmark",
    "niat.argos_evolucao_temporal_setor",
    "niat.argos_evolucao_temporal_empresa",
    "niat.argos_anomalias_setoriais",
    "niat.argos_alertas_empresas"
]

VIEWS_AUXILIARES = [
    "niat.vw_dashboard_setores",
    "niat.vw_dashboard_empresas",
    "niat.vw_analise_volatilidade",
    "niat.vw_relacao_icms_pagamentos"
]

OUTPUT_DIR = "./data-schema"

# =============================================================================
# FUNÇÕES
# =============================================================================

def criar_diretorio_output(base_dir):
    """Cria estrutura de diretórios para os data-schemas."""
    Path(base_dir).mkdir(parents=True, exist_ok=True)
    Path(f"{base_dir}/originais").mkdir(exist_ok=True)
    Path(f"{base_dir}/intermediarias").mkdir(exist_ok=True)
    Path(f"{base_dir}/views").mkdir(exist_ok=True)
    print(f"✅ Diretórios criados em: {base_dir}")


def formatar_nome_arquivo(tabela_completa):
    """Converte nome da tabela para nome de arquivo."""
    nome_tabela = tabela_completa.split('.')[-1]
    return f"{nome_tabela}.md"


def executar_describe_formatted(spark, tabela):
    """Executa DESCRIBE FORMATTED e retorna resultado como Pandas DataFrame."""
    try:
        print(f"  📋 Executando DESCRIBE FORMATTED {tabela}...")
        df = spark.sql(f"DESCRIBE FORMATTED {tabela}")
        return df.toPandas()
    except Exception as e:
        print(f"  ❌ ERRO ao descrever {tabela}: {e}")
        return None


def executar_select_sample(spark, tabela, limit=10):
    """Executa SELECT * LIMIT e retorna resultado como Pandas DataFrame."""
    try:
        print(f"  📊 Executando SELECT * FROM {tabela} LIMIT {limit}...")
        df = spark.sql(f"SELECT * FROM {tabela} LIMIT {limit}")
        return df.toPandas()
    except Exception as e:
        print(f"  ❌ ERRO ao fazer SELECT de {tabela}: {e}")
        return None


def gerar_markdown_dataschema(tabela, df_describe, df_sample):
    """Gera conteúdo markdown do data-schema."""

    nome_tabela = tabela.split('.')[-1]
    schema_nome = tabela.split('.')[0]

    md = f"""# Data Schema: {tabela}

**Gerado em:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

---

## 📋 DESCRIBE FORMATTED

```sql
DESCRIBE FORMATTED {tabela};
```

### Resultado:

"""

    if df_describe is not None and not df_describe.empty:
        md += "```\n"
        md += df_describe.to_string(index=False, max_colwidth=100)
        md += "\n```\n"
    else:
        md += "_Nenhum resultado disponível_\n"

    md += f"""

---

## 📊 SAMPLE DATA (LIMIT 10)

```sql
SELECT * FROM {tabela} LIMIT 10;
```

### Resultado:

"""

    if df_sample is not None and not df_sample.empty:
        md += "```\n"
        md += df_sample.to_string(index=False, max_colwidth=50)
        md += "\n```\n"

        md += f"""

### Informações Adicionais:

- **Total de colunas:** {len(df_sample.columns)}
- **Colunas:** {', '.join(df_sample.columns.tolist())}
- **Registros retornados:** {len(df_sample)}

"""
    else:
        md += "_Nenhum resultado disponível_\n"

    md += """
---

## 📝 Notas

- Este arquivo foi gerado automaticamente
- Utilize este schema como referência para análises e desenvolvimento

"""

    return md


def salvar_arquivo(conteudo, caminho):
    """Salva conteúdo em arquivo."""
    try:
        with open(caminho, 'w', encoding='utf-8') as f:
            f.write(conteudo)
        print(f"  ✅ Salvo: {caminho}")
        return True
    except Exception as e:
        print(f"  ❌ ERRO ao salvar {caminho}: {e}")
        return False


print("✅ Funções carregadas com sucesso!")

## 🚀 PASSO 3: Executar Geração

### Opções:

**Opção A:** Gerar apenas tabelas originais e intermediárias (recomendado)

**Opção B:** Gerar incluindo views auxiliares

Execute a célula correspondente à opção desejada:

In [ ]:
# =============================================================================
# OPÇÃO A: SEM VIEWS (RECOMENDADO)
# =============================================================================

print("=" * 80)
print("GERADOR DE DATA-SCHEMAS - ARGOS SETORES")
print("=" * 80)
print(f"Iniciado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

criar_diretorio_output(OUTPUT_DIR)

total = 0
sucesso = 0
erros = 0

# Processar tabelas originais
print("\n" + "=" * 80)
print("PROCESSANDO TABELAS ORIGINAIS (ODS)")
print("=" * 80)

for tabela in TABELAS_ORIGINAIS:
    total += 1
    print(f"\n[{total}] Processando: {tabela}")
    
    df_desc = executar_describe_formatted(spark, tabela)
    df_sample = executar_select_sample(spark, tabela, limit=10)
    
    if df_desc is not None or df_sample is not None:
        md_content = gerar_markdown_dataschema(tabela, df_desc, df_sample)
        nome_arquivo = formatar_nome_arquivo(tabela)
        caminho = f"{OUTPUT_DIR}/originais/{nome_arquivo}"
        
        if salvar_arquivo(md_content, caminho):
            sucesso += 1
        else:
            erros += 1
    else:
        erros += 1
        print(f"  ⚠️ Pulando {tabela} (sem dados)")

# Processar tabelas intermediárias
print("\n" + "=" * 80)
print("PROCESSANDO TABELAS INTERMEDIÁRIAS (NIAT)")
print("=" * 80)

for tabela in TABELAS_INTERMEDIARIAS:
    total += 1
    print(f"\n[{total}] Processando: {tabela}")
    
    df_desc = executar_describe_formatted(spark, tabela)
    df_sample = executar_select_sample(spark, tabela, limit=10)
    
    if df_desc is not None or df_sample is not None:
        md_content = gerar_markdown_dataschema(tabela, df_desc, df_sample)
        nome_arquivo = formatar_nome_arquivo(tabela)
        caminho = f"{OUTPUT_DIR}/intermediarias/{nome_arquivo}"
        
        if salvar_arquivo(md_content, caminho):
            sucesso += 1
        else:
            erros += 1
    else:
        erros += 1
        print(f"  ⚠️ Pulando {tabela} (sem dados)")

# Relatório final
print("\n" + "=" * 80)
print("RELATÓRIO FINAL")
print("=" * 80)
print(f"Total de tabelas processadas: {total}")
print(f"✅ Sucesso: {sucesso}")
print(f"❌ Erros: {erros}")
print(f"\n📁 Arquivos salvos em: {OUTPUT_DIR}")
print("\n" + "=" * 80)
print("✅ GERAÇÃO CONCLUÍDA!")
print("=" * 80)

In [ ]:
# =============================================================================
# OPÇÃO B: COM VIEWS AUXILIARES
# =============================================================================
# Descomente e execute esta célula se quiser incluir as views auxiliares

# print("\n" + "=" * 80)
# print("PROCESSANDO VIEWS AUXILIARES")
# print("=" * 80)

# for tabela in VIEWS_AUXILIARES:
#     total += 1
#     print(f"\n[{total}] Processando: {tabela}")
    
#     df_desc = executar_describe_formatted(spark, tabela)
#     df_sample = executar_select_sample(spark, tabela, limit=10)
    
#     if df_desc is not None or df_sample is not None:
#         md_content = gerar_markdown_dataschema(tabela, df_desc, df_sample)
#         nome_arquivo = formatar_nome_arquivo(tabela)
#         caminho = f"{OUTPUT_DIR}/views/{nome_arquivo}"
        
#         if salvar_arquivo(md_content, caminho):
#             sucesso += 1
#         else:
#             erros += 1
#     else:
#         erros += 1
#         print(f"  ⚠️ Pulando {tabela} (sem dados)")

## 📊 PASSO 4: Verificar Resultados

Liste os arquivos gerados:

In [ ]:
import os

print("📁 ARQUIVOS GERADOS:")
print("=" * 80)

for root, dirs, files in os.walk(OUTPUT_DIR):
    level = root.replace(OUTPUT_DIR, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f"{subindent}📄 {file}")

---

## ✅ Concluído!

Os arquivos de data-schema foram gerados em:

```
./data-schema/
├── originais/          # Tabelas ODS (usr_sat_ods.*)
├── intermediarias/     # Tabelas NIAT (niat.argos_*)
└── views/              # Views auxiliares (niat.vw_*)
```

Cada arquivo contém:
- DESCRIBE FORMATTED completo
- SELECT com 10 registros de exemplo
- Informações sobre colunas e tipos

---